# Home exam GR
Kandidatnr: 10018

In [1]:
from sympy import MatrixSymbol, Matrix, Array, pprint
from sympy import symbols, diff, cos, sin, simplify, Rational
from sympy.core.symbol import Symbol
from sympy import pi

import numpy as np
import sympy as sp
from IPython.display import display, Latex

### Utilities
Misc. tensor operations

In [2]:
def get_g_inv(g):
    return np.array(Matrix(g)**(-1))

In [3]:
def INDX(i, place, num_indx):
    """
    Acceses an index at 'place' for 'num_indx' order tensor
    T_(a0 ... âp ... an-1) = T[INDX(i, place=p, num_indx=n)] = T[:,...<-p-> , i, :,...<-(n-p-1)->]
    """
    indx = []
    assert place<num_indx
    for j in range(num_indx):
        if place==j: indx.append(i)
        else: indx.append(slice(None))
    return tuple(indx)

def get_g_inv(g):
    return np.array(Matrix(g)**(-1))

In [4]:
def contract(T, g=None, g_inv=None, num_indx=2, upper=1, indx=(0, 1)):
    """
    contracts indecies indx=(a_p, a_q) on tensor T with 'num_indx', 
    'upper' of whom are upper. If upper=0, all indecies are assumed lower.
    With indx=(a_k, a_l), upper=n, num_indx=n+m, this gives
    T^(a_0...a_n-1)_(a_n...a_n+m-1) -> T^(a_0...a_k=a...a_n-1)_(a_n...a_k...a_n+m-1),
    with the necesarry metric. If wrong metric is given, this wil throw error.
    """
    assert indx[0] < indx[1]  # we have to know if the index to the left dissapears
    dim = np.shape(T)[0]
    a = (indx[0] < upper) + (indx[1] < upper) # number of upper indecies to be contracted
    if a==2: g0 = g # two upper
    elif a==0: g0 = g_inv # two lower
    else: g0 = np.identity(dim, dtype=Rational)

    Tc = Rational(0) * np.ones((T.shape)[:-2], dtype=Rational)
    for i in range(dim):
        for j in range(dim):
            Tc += g0[i, j] * (T[INDX(i, indx[0], num_indx)])[INDX(j, indx[1] - 1, num_indx - 1)]

    return Tc

In [5]:
def raise_indx(T, g_inv, indx, num_indx):
    """
    Raise index 'indx' of a tensor T with 'num_indx' indices.
    """
    dim = np.shape(T)[0]
    Tu = np.zeros_like(T)
    for i in range(dim):
        I = INDX(i, indx, num_indx)
        for j in range(dim):
            J = INDX(j, indx, num_indx)
            Tu[I] += g_inv[i, j] * T[J]
    return Tu

def lower_indx(T, g, indx, num_indx):
    return raise_indx(T, g, indx, num_indx)

In [6]:
def print_christoffel(C, var):
    """ A function for dsiplaying christoffels symbols """
    output = []
    for i in range(len(var)):
        txt = "$$"
        txt += "\Gamma^" + sp.latex(var[i]) + "_{\\alpha \\beta} ="
        txt += sp.latex(Matrix(C[i]))
        txt += "$$"
        output.append(display(Latex(txt)))
    return output

def print_tensor(T, name="T"):
    return display(Latex("$$" + name + "=" + sp.latex(Matrix(T)) +"$$"))

def print_scalar(S, name="S"):
    return display(Latex("$$" + name + "=" + sp.latex(S) +"$$"))

## Geometry

In [7]:
def Christoffel(g, g_inv, var):
    """ 
    Work out the christoffel symbols, given a metric an its variables 
    Γ^i_jk = C[i, j, k]
    """
    dim = len(var)
    C = np.zeros((dim, dim, dim), dtype=Symbol)
    for i in range(dim):
        for j in range(dim):
            for k in range(dim):
                for m in range(dim):
                    C[i, j, k] += Rational(1, 2) * (g_inv)[i, m] * (
                        diff(g[m, k], var[j])
                        + diff(g[m, j], var[k])
                        - diff(g[k, j], var[m])
                        )
    return C

$
R^\alpha_{\beta \rho \sigma} = \partial_\rho \Gamma^\alpha_{\beta \sigma} -\partial_\sigma \Gamma^\alpha_{\beta \rho} +\Gamma^\alpha_{\kappa \rho}\Gamma^\kappa_{\beta \sigma} - \Gamma^\alpha_{\kappa \sigma}\Gamma^\kappa_{\beta \rho}
$

    R[a, b, r, s] = diff(C[a, b, s], r) - diff(C[a, b, r], s) + C[a, k, r]*C[k, b, s] * C[a, k, s]*C[k, b, r]

In [8]:
def Riemann_tensor(C, var):
    """ 
    Riemann_tensor(Christoffel_symbols, (x_1, ...)) = R[i, j, k, l] = R^i_jkl
    Compute the Riemann tensor from the Christoffel symbols 
    """
    dim = len(var)
    R = np.zeros([dim] * 4, dtype=Symbol)
    indx = [(i, j, k, l)
        for i in range(dim)
        for j in range(dim)
        for k in range(dim)
        for l in range(dim)
    ]
    for (a, b, r, s) in indx:
        R[a, b, r, s] += diff(C[a, b, s], var[r]) - diff(C[a, b, r], var[s])
        for k in range(dim):
            R[a, b, r, s] += C[a, k, r] * C[k, b, s] - C[a, k, s] * C[k, b, r]
    return R

# Exercise b)

In [9]:
t, r, th, ph = symbols("t, r, \\theta, \phi")
x1 = r * cos(ph) * sin(th)
x2 = r * sin(ph) * sin(th)
x3 = r * cos(th)

one = Rational(1)
eta = sp.diag(one, -one, -one, -one)
var = (t, r, th, ph)
J = Matrix([t, x1, x2, x3]).jacobian(var)
g = np.array(simplify(J.T *eta* J))

A = sp.Function("A", real=True)(r)
B = sp.Function("B", real=True)(r)
g[0, 0] *= A
g[1, 1] *= B
g_inv = get_g_inv(g)

Latex("$$" + sp.latex(Matrix(g))  + sp.latex(Matrix(g_inv)) +"$$")

<IPython.core.display.Latex object>

In [10]:
C = Christoffel(g, g_inv, var)
c = print_christoffel(C, var)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [11]:
Rie = Riemann_tensor(C, var)
print_tensor(Rie[0, 1])

<IPython.core.display.Latex object>

In [12]:
Ricci = -contract(Rie, num_indx=4, upper=1, indx=(0, 2))

In [13]:
for i in range(len(Ricci)):
    display(Latex("$$ R_{ "+ str(i)+str(i) +"}=" + sp.latex(Ricci[i,i]) + "$$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Exercise c)

In [14]:
E1, E2, E3, B1, B2, B3 = symbols("E_1, E_2, E_3, B_1, B_2, B_3")

F = np.array([
    [0,     E1,    E2,    E3],
    [-E1,    0,      B3,    -B2],
    [-E2,    -B3,     0,      B1],
    [-E3,    B2,    -B1,     0 ]
])

print_tensor(F, "F_{\mu \\nu}")

<IPython.core.display.Latex object>

In [15]:
Fud = raise_indx(F, g_inv, 0, 2)
Fdu = raise_indx(F, g_inv, 1, 2)
Fuu = raise_indx(Fud, g_inv, 1, 2)

print_tensor(Fud, "F^\mu_{\,\,\,\,\\nu}")
print_tensor(Fdu, "F^{\,\,\\nu}_\\mu")
print_tensor(Fuu, "F^{\mu \\nu}")

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

# Exercise d)

In [16]:
Er = symbols("E_r")

F0 = np.array([
    [0, Er, 0, 0], 
    [-Er, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]
])

F0ud = raise_indx(F0, g_inv, 0, 2)
F0du = raise_indx(F0, g_inv, 1, 2)
F0uu = raise_indx(F0ud, g_inv, 1, 2)

print_tensor(F0ud, "F^\mu_{\,\,\,\,\\nu}")
print_tensor(F0du, "F^{\,\,\\nu}_\\mu")
print_tensor(F0uu, "F^{\mu \\nu}")

Tu = -F0uu @ F0ud.T + Rational(1, 4) * g_inv * contract(F0.T@ F0uu)
T = lower_indx(lower_indx(Tu, g, 0, 2), g, 1, 2)

print_tensor(Tu, "T^{\mu \\nu}")
print_tensor(T, "T_{\mu \\nu}")
print_scalar(contract(T, g_inv=g_inv, upper=0), "T_\mu^\mu")

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [17]:
Q = symbols("Q")
k = symbols("k")
# k =  kappa *( Q / ( Rational(4) pi))**2
E = sp.sqrt(A*B) / r**2
T0 = Matrix(T).subs(Er, E)
eq = []
for i in range(len(Ricci)):
    eq.append(Ricci[i, i] + k*T0[i, i])
    display(Latex("$$" + sp.latex(eq[i]) + "= 0 $$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [18]:
simplify(-B*r*(B * eq[0] + A * eq[1]))

A(r)*Derivative(B(r), r) + B(r)*Derivative(A(r), r)

# Exercise f)

In [19]:
eq0 = [a.subs(B, 1/A) for a in eq]

for i in range(len(Ricci)):
    display(Latex("$$" + sp.latex(simplify(eq0[i])) + "= 0 $$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [20]:
sp.dsolve(eq0[2])

Eq(A(r), C1/r + k/(2*r**2) + 1)

# Excercise g)

In [21]:
M = symbols("M")

g0 = Matrix(g).subs(B, 1/A)
g0 = Matrix(g0).subs(A, 1 - 2*M/r + k/(2*r**2))
g0_inv = g0**(-1)

print_tensor(g0, "g")
print_tensor(g0**(-1), "g^{-1}")
C0 = Christoffel(g0, g0_inv, var)
print_christoffel(C0, var)
Rie0 = Riemann_tensor(C0, var)
Ricci0 = -contract(Rie0, upper=1, num_indx=4, indx=(0, 2))
print_tensor(simplify(Ricci0), "R_t")
R0 = contract(Ricci0, upper=0, g_inv=g0_inv)
R0 = simplify(R0)
print_scalar(R0, "R_s")

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [22]:
Ru = raise_indx(raise_indx(raise_indx(Rie0, g0_inv, 1, 4), g0_inv, 2, 3), g0_inv, 3, 4)
Rl = lower_indx(Rie0, g0, 0, 4)


simplify(sum(sum(sum(sum(Ru * Rl)))))

2*(24*M**2*r**2 - 24*M*k*r + 7*k**2)/r**8